In [191]:
class game:
    #=== constructor
    def __init__(self):
        import numpy as np
        self.board = np.zeros([9], dtype='int')
    #=== looking for pattern (with transformations)
    def match_pattern(self, pattern):
        #-- function looking for pattern within vector
        def find_pattern(target, pattern):
            match = True
            for index in range(pattern.shape[0]):
                if pattern[index] != 0:
                    if pattern[index] != target[index]:
                        match = False
                        break
            return match
        #-- for all possible transformations
        for flipud in [True,False]:
            for fliplr in [True,False]:
                for transpose in [True,False]:
                    temp = self.board.reshape([3,3]) # reshape to 2-D array
                    #-- apply tranformations
                    if flipud: temp = np.flipud(temp)
                    if fliplr: temp = np.fliplr(temp)
                    if transpose: temp = temp.T
                    temp = temp.reshape(9)          # reshape back to vector
                    if find_pattern(temp, pattern): return True
        return False
    #=== doing a move
    def move(self, field):
        if self.board[field] == 0 :
            if np.count_nonzero(self.board) % 2 == 0:
                self.board[field] = 1
            else:
                self.board[field] = 2
        else:
            raise Exception('Forbidden move!')
    #=== looking for win
    def ifWin(self):
        pass
    #=== displaying actual state of the game        
    def show_board(self):
        inrow = 1
        print('–––––')
        for field in self.board :
            #=== convert ints to characters
            if   field == 0: sign = '•'
            elif field == 1: sign = 'X'
            else:            sign = 'O'
            #=== display board
            if inrow == 3:
                print(sign)
                inrow = 1
            else:
                print(sign, end=" ")
                inrow += 1
        print('–––––')

In [192]:
#=== run test
g1 = game()
g1.move(0)
g1.move(1)
#-- test
g1.show_board()
g1.match_pattern(np.array([0,0,0,0,0,0,0,2,1]))

–––––
X O •
• • •
• • •
–––––


True